# Data mining

## Import and directory setup

In [ ]:
# # Create a 3.11 venv (Homebrew’s python@3.11)
# /opt/homebrew/opt/python@3.11/bin/python3.11 -m venv .venv
# source .venv/bin/activate
# python --version  # should show 3.11.x
# pip install --upgrade pip
# pip install -r app/requirements.txt


In [ ]:
# %pip install -U --force-reinstall --no-cache-dir \
#     numpy==2.0.2 \
#     pandas==2.2.2 \
#     requests==2.32.3 \
#     pyyaml==6.0.2 \
#     yfinance==0.2.43 \
#     scikit-learn==1.5.1 \
#     statsmodels==0.14.2 \
#     matplotlib==3.9.0
# %pip install -q pyalex pandas numpy requests pyyaml yfinance scikit-learn \
#     statsmodels matplotlib pandas_datareader seaborn


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import numpy as np
import requests, time, json, io, datetime as dt
from pathlib import Path
import yaml
from itertools import chain
from datetime import datetime
import pyalex
from pyalex import Topics, Works, config
import yfinance as yf
from pandas_datareader import data as pdr
import csv

In [3]:
# Data directories
DATA_DIR = Path("data")
RAW_OPENALEX_DIR = DATA_DIR / "raw" / "openalex"
RAW_OPENALEX_COUNTS_DIR = DATA_DIR / "raw" / "openalex_counts"
RAW_PRICES_DIR = DATA_DIR / "raw" / "prices"
PROCESSED_DIR = DATA_DIR / "processed"

for p in [RAW_OPENALEX_DIR, RAW_OPENALEX_COUNTS_DIR, RAW_PRICES_DIR, PROCESSED_DIR]:
    p.mkdir(parents=True, exist_ok=True)

START_DATE = "2025-01-01"
END_DATE   = "2025-08-31"

In [4]:
with open("sector_map.yml", "r") as f:
    SECTOR_MAP = yaml.safe_load(f)

SECTOR_MAP

{'Biotech': {'tickers': ['XBI', 'IBB'],
  'keywords': ['biotech',
   'crispr',
   'gene editing',
   'oncology',
   'immunotherapy',
   'sequencing',
   'rna',
   'mrna']},
 'Semiconductors': {'tickers': ['SOXX', 'SMH'],
  'keywords': ['semiconductor',
   'lithography',
   'euv',
   'transistor',
   'fab',
   'wafer',
   'chiplet',
   'gpu',
   'asic']}}

## Fetch publication data

In [5]:
# REQUIRED: use a real email (OpenAlex "polite pool" = faster, fewer errors)
pyalex.config.email = "xuexd0211@g.ucla.edu"   # <-- change this
# Retries recommended
config.max_retries = 1
config.retry_backoff_factor = 0.5
config.retry_http_codes = [429, 500, 502, 503, 504]


In [18]:
# Search topics by display_name/description/keywords
topics = Topics().search("Semiconductor device").get()
topics[:3]  # inspect; grab the "id" fields like 'T12345'
#print(len(topics))

[{'id': 'https://openalex.org/T10099',
  'display_name': 'GaN-based semiconductor devices and materials',
  'description': 'This cluster of papers focuses on the first-principles calculations, properties, and applications of III-nitride semiconductors, particularly Gallium Nitride (GaN) and its alloys. It covers topics such as defects and impurities, band parameters, high-power light-emitting diodes (LEDs), AlGaN/GaN HEMTs, nanowires, UV LEDs, and their applications in solid-state lighting.',
  'keywords': ['III-Nitrides',
   'Semiconductors',
   'Light-Emitting Diodes',
   'GaN',
   'AlGaN/GaN HEMTs',
   'Defects and Impurities',
   'Solid-State Lighting',
   'Nanowires',
   'UV LEDs',
   'Band Parameters'],
  'ids': {'openalex': 'https://openalex.org/T10099',
   'wikipedia': 'https://en.wikipedia.org/wiki/III-nitride_semiconductor'},
  'subfield': {'id': 'https://openalex.org/subfields/3104',
   'display_name': 'Condensed Matter Physics'},
  'field': {'id': 'https://openalex.org/fiel

In [11]:
T = "T12111"  # replace with your topic id
rows = Works().filter(topics={"id": T}).select(["id","title","publication_date","topics"]).get()
len(rows), rows[0]

(25,
 {'id': 'https://openalex.org/W2133059825',
  'title': 'A Threshold Selection Method from Gray-Level Histograms',
  'publication_date': '1979-01-01',
  'topics': [{'id': 'https://openalex.org/T10057',
    'display_name': 'Face and Expression Recognition',
    'score': 0.4799,
    'subfield': {'id': 'https://openalex.org/subfields/1707',
     'display_name': 'Computer Vision and Pattern Recognition'},
    'field': {'id': 'https://openalex.org/fields/17',
     'display_name': 'Computer Science'},
    'domain': {'id': 'https://openalex.org/domains/3',
     'display_name': 'Physical Sciences'}},
   {'id': 'https://openalex.org/T12111',
    'display_name': 'Industrial Vision Systems and Defect Detection',
    'score': 0.4697,
    'subfield': {'id': 'https://openalex.org/subfields/2209',
     'display_name': 'Industrial and Manufacturing Engineering'},
    'field': {'id': 'https://openalex.org/fields/22',
     'display_name': 'Engineering'},
    'domain': {'id': 'https://openalex.org/do

In [29]:
# Reuse the same sector → Topic display name mapping you set earlier
SECTOR_PRIMARY_TOPIC_NAME = {
    "Biotech": "Biotechnology",
    "Semiconductors": "Semiconductor device",
}

def resolve_topic_id(display_name: str, take_first_if_no_exact=True):
    cand = Topics().search(display_name).get()
    if not cand:
        raise ValueError(f"No Topic found for query: {display_name!r}")
    for t in cand:
        if t.get("display_name","").strip().lower() == display_name.strip().lower():
            return t["id"], t
    if take_first_if_no_exact:
        return cand[0]["id"], cand[0]
    raise ValueError(f"No exact Topic display_name match for {display_name!r}. Candidates: {[c.get('display_name') for c in cand[:5]]}")

def fetch_works_with_topics(topic_id: str,
                            start: str,
                            end: str,
                            select=("id","publication_date","topics"),
                            per_page=200,
                            n_max=None,
                            primary_only=True):
    """
    Pull works for a Topic within [start, end], selecting id, publication_date, and topics[].
    primary_only=True -> filter by primary_topic.id (most precise);
    primary_only=False -> filter by topics.id (broader net).
    """
    base = (Works()
            .filter(from_publication_date=start, to_publication_date=end)
            .select(["id","publication_date","topics"]))
    q = base.filter(primary_topic={"id": topic_id}) if primary_only else base.filter(topics={"id": topic_id})
    pages = q.paginate(per_page=per_page, n_max=n_max)
    return list(chain(*pages))

def normalize_topics_list(r):
    """
    Extract a clean list of topic display names from a work record.
    OpenAlex 'topics' is a list of objects; we take the 'display_name'.
    """
    ts = r.get("topics") or []
    names = []
    for t in ts:
        name = (t.get("display_name") or "").strip()
        if name:
            names.append(name)
    # de-duplicate within a single work
    return sorted(set(names))

def build_topic_counts_tables(records: list):
    """
    Build topic count tables + Top-5 wide tables with:
      - daily_topic_counts: [date, topic, count]
      - daily_top_long:     [date, rank, topic, count]
      - weekly_topic_counts:[week, topic, count]
      - weekly_top_long:    [week, rank, topic, count]
      - daily_top_wide:     [date, top1..top5, top1_count..top5_count, total_publications,
                             top1_share..top5_share, top_list]
      - weekly_top_wide:    [week, ... same fields ...]
    """
    import pandas as pd

    def _empty():
        empty_daily_counts = pd.DataFrame(columns=["date","topic","count"])
        empty_daily_top    = pd.DataFrame(columns=["date","rank","topic","count"])
        empty_weekly_counts= pd.DataFrame(columns=["week","topic","count"])
        empty_weekly_top   = pd.DataFrame(columns=["week","rank","topic","count"])
        empty_daily_wide   = pd.DataFrame(columns=[
            "date","top1","top2","top3","top4","top5",
            "top1_count","top2_count","top3_count","top4_count","top5_count",
            "total_publications","top1_share","top2_share","top3_share","top4_share","top5_share",
            "top_list"
        ])
        empty_weekly_wide  = pd.DataFrame(columns=[
            "week","top1","top2","top3","top4","top5",
            "top1_count","top2_count","top3_count","top4_count","top5_count",
            "total_publications","top1_share","top2_share","top3_share","top4_share","top5_share",
            "top_list"
        ])
        return (empty_daily_counts, empty_daily_top,
                empty_weekly_counts, empty_weekly_top,
                empty_daily_wide, empty_weekly_wide)

    if not records:
        return _empty()

    # --- normalize works to [id, date, topics_list]
    rows = []
    for r in records:
        d = r.get("publication_date")
        if not d:
            continue
        try:
            date = pd.to_datetime(d).normalize()
        except Exception:
            continue
        topics_list = normalize_topics_list(r)
        rows.append((r.get("id"), date, topics_list))
    works_df = pd.DataFrame(rows, columns=["id","date","topics_list"]).drop_duplicates(subset=["id"])

    exploded = (works_df
                .explode("topics_list")
                .dropna(subset=["topics_list"])
                .rename(columns={"topics_list":"topic"}))

    if exploded.empty:
        return _empty()

    # --- Daily counts (by date, topic)
    daily_topic_counts = (exploded
                          .groupby(["date","topic"], as_index=False)
                          .size()
                          .rename(columns={"size":"count"})
                          .sort_values(["date","count","topic"], ascending=[True, False, True]))

    # Total publications per day (sum across topics after de-dup per work)
    # Since we exploded topics, sum of counts per day equals number of topic-tags that day, not unique works.
    # For total unique works per day, use works_df:
    daily_total_unique = works_df.groupby("date", as_index=False).size().rename(columns={"size":"total_publications"})

    # Top-5 per day
    daily_topN = (daily_topic_counts
                  .sort_values(["date","count","topic"], ascending=[True, False, True])
                  .groupby("date", as_index=False)
                  .head(5))
    daily_topN["rank"] = (daily_topN.groupby("date")["count"]
                          .rank(method="first", ascending=False)
                          .astype(int))

    def _topN_to_wide(topN: pd.DataFrame, time_col: str, totals_df: pd.DataFrame) -> pd.DataFrame:
        topN = topN.copy()
        topN["k_topic"] = "top" + topN["rank"].astype(int).astype(str)
        topN["k_count"] = topN["k_topic"] + "_count"

        topic_wide = (topN.pivot(index=time_col, columns="k_topic", values="topic")
                           .reset_index().rename_axis(None, axis=1))
        count_wide = (topN.pivot(index=time_col, columns="k_count", values="count")
                           .reset_index().rename_axis(None, axis=1))

        wide = topic_wide.merge(count_wide, on=time_col, how="outer")
        wide = wide.merge(totals_df, left_on=time_col, right_on=time_col, how="left")

        # ensure all columns exist
        for col in ["top1","top2","top3","top4","top5",
                    "top1_count","top2_count","top3_count","top4_count","top5_count",
                    "total_publications"]:
            if col not in wide.columns:
                wide[col] = pd.NA

        # shares = topN_count / total_unique_works_that_day
        for i in range(1, 6):
            ccol = f"top{i}_count"
            scol = f"top{i}_share"
            wide[scol] = (wide[ccol] / wide["total_publications"]).astype(float)

        # pipe-joined list
        wide["top_list"] = (wide[["top1","top2","top3","top4","top5"]]
                            .fillna("")
                            .apply(lambda r: "|".join([t for t in r if t]), axis=1))

        # order
        ordered_cols = [time_col, "top1","top2","top3","top4","top5",
                        "top1_count","top2_count","top3_count","top4_count","top5_count",
                        "total_publications","top1_share","top2_share","top3_share","top4_share","top5_share",
                        "top_list"]
        return wide[ordered_cols].sort_values(time_col).reset_index(drop=True)

    daily_top_long = (daily_topN[["date","rank","topic","count"]]
                      .sort_values(["date","rank"])
                      .reset_index(drop=True))
    daily_top_wide = _topN_to_wide(daily_topN, "date", daily_total_unique)

    # --- Weekly counts
    weekly_topic_counts = (daily_topic_counts
                           .assign(week=pd.to_datetime(daily_topic_counts["date"])
                                   .dt.to_period("W-SUN").dt.end_time.dt.normalize())
                           .groupby(["week","topic"], as_index=False)["count"].sum()
                           .sort_values(["week","count","topic"], ascending=[True, False, True]))

    # total unique works per week (from works_df)
    weekly_total_unique = (works_df
                           .assign(week=works_df["date"].dt.to_period("W-SUN").dt.end_time.dt.normalize())
                           .groupby("week", as_index=False)
                           .size()
                           .rename(columns={"size":"total_publications"}))

    weekly_topN = (weekly_topic_counts
                   .groupby("week", as_index=False)
                   .head(5)
                   .copy())
    weekly_topN["rank"] = (weekly_topN.groupby("week")["count"]
                           .rank(method="first", ascending=False)
                           .astype(int))
    weekly_top_long = (weekly_topN[["week","rank","topic","count"]]
                       .sort_values(["week","rank"])
                       .reset_index(drop=True))
    weekly_top_wide = _topN_to_wide(weekly_topN, "week", weekly_total_unique)

    return (daily_topic_counts, daily_top_long,
            weekly_topic_counts, weekly_top_long,
            daily_top_wide, weekly_top_wide)

def save_topic_outputs(sector: str,
                       daily_counts: pd.DataFrame,
                       daily_top_long: pd.DataFrame,
                       weekly_counts: pd.DataFrame,
                       weekly_top_long: pd.DataFrame,
                       daily_top_wide: pd.DataFrame,
                       weekly_top_wide: pd.DataFrame):
    outdir = PROCESSED_DIR / "topics"
    outdir.mkdir(parents=True, exist_ok=True)

    # Force quotes on **all** fields to protect commas in topic names
    daily_counts.to_csv(outdir / f"daily_topic_counts_{sector}.csv",
                        index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    weekly_counts.to_csv(outdir / f"weekly_topic_counts_{sector}.csv",
                        index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    daily_top_long.to_csv(outdir / f"daily_top5_{sector}.csv",
                        index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    weekly_top_long.to_csv(outdir / f"weekly_top5_{sector}.csv",
                        index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    daily_top_wide.to_csv(outdir / f"daily_top5_wide_{sector}.csv",
                        index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    weekly_top_wide.to_csv(outdir / f"weekly_top5_wide_{sector}.csv",
                        index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)

def build_topic_features_for_sector(sector: str, start: str, end: str, primary_only=True):
    """End-to-end for one sector: resolve topic → fetch works → build & save topic features."""
    # Resolve Topic ID
    topic_name = SECTOR_PRIMARY_TOPIC_NAME.get(sector)
    if not topic_name:
        raise KeyError(f"No Topic display name configured for sector {sector!r}. Add it to SECTOR_PRIMARY_TOPIC_NAME.")
    topic_id, topic_obj = resolve_topic_id(topic_name)
    print(f"[{sector}] Using Topic: {topic_obj.get('display_name')} (id={topic_id})")

    # Fetch works with related topics field
    recs = fetch_works_with_topics(topic_id, start, end, select=("id","publication_date","topics"), primary_only=primary_only)

    # Build tables
    daily_counts, daily_top_long, weekly_counts, weekly_top_long, daily_top_wide, weekly_top_wide = build_topic_counts_tables(recs)

    # Save
    save_topic_outputs(sector, daily_counts, daily_top_long, weekly_counts, weekly_top_long, daily_top_wide, weekly_top_wide)

    # Return compact daily & weekly “categorical feature” tables to merge later
    # daily_top_wide has columns: date, top1..top5, top_list
    # weekly_top_wide has: week, top1..top5, top_list
    return daily_top_wide, weekly_top_wide

# ---- Run for all sectors; produce daily/weekly top-5 categorical features
topic_daily_features = {}
topic_weekly_features = {}
for sector in SECTOR_MAP.keys():
    d_wide, w_wide = build_topic_features_for_sector(sector, START_DATE, END_DATE, primary_only=True)
    topic_daily_features[sector] = d_wide
    topic_weekly_features[sector] = w_wide
    print(f"[{sector}] topic features built: daily={d_wide.shape}, weekly={w_wide.shape}")


[Biotech] Using Topic: Bacterial Genetics and Biotechnology (id=https://openalex.org/T10120)
[Biotech] topic features built: daily=(186, 18), weekly=(35, 18)
[Semiconductors] Using Topic: GaN-based semiconductor devices and materials (id=https://openalex.org/T10099)
[Semiconductors] topic features built: daily=(225, 18), weekly=(35, 18)


In [23]:
### number of unique categories
sector = 'Biotech'
semi_top5 = pd.read_csv(PROCESSED_DIR / "topics" / f"daily_top5_{sector}.csv")
semi_top5['topic'].nunique()

74

In [25]:
sector = 'Semiconductors'
semi_top5 = pd.read_csv(PROCESSED_DIR / "topics" / f"daily_top5_{sector}.csv")
semi_top5['topic'].nunique()

60

## Fetch stock price

In [34]:
RAW_PRICES_DIR.mkdir(parents=True, exist_ok=True)

def fetch_stooq_ticker(ticker: str, start: str, end: str) -> pd.DataFrame:
    """
    Fetch OHLCV from Stooq for a single ticker, robust to common quirks.
    Returns columns: date, close, volume, ticker (ascending by date).
    """
    tried = []
    for sym in (ticker, f"{ticker}.US", ticker.lower(), f"{ticker.lower()}.us"):
        tried.append(sym)
        try:
            df = pdr.DataReader(sym, "stooq", start=start, end=end)
            if df is not None and not df.empty:
                # Stooq returns descending; sort ascending and normalize cols
                df = df.sort_index()
                df = df.reset_index().rename(columns={"Date":"date","Close":"close","Volume":"volume"})
                out = df[["date","close","volume"]].copy()
                out["ticker"] = ticker
                return out
        except Exception:
            continue
    raise RuntimeError(f"Stooq fetch failed for {ticker}. Tried: {tried}")

# (Re)download & save each ticker (you can skip if you already have clean per-ticker CSVs)
for sector, meta in SECTOR_MAP.items():
    for t in meta["tickers"]:
        try:
            df = fetch_stooq_ticker(t, START_DATE, END_DATE)
            (RAW_PRICES_DIR / f"{t}.csv").write_text(df.to_csv(index=False))
            print(f"[{sector}] saved {t}: {len(df)} rows {df['date'].min().date()} → {df['date'].max().date()}")
        except Exception as e:
            print(f"[WARN] {t}: {e}")


[Biotech] saved XBI: 165 rows 2025-01-02 → 2025-08-29
[Biotech] saved IBB: 165 rows 2025-01-02 → 2025-08-29
[Semiconductors] saved SOXX: 165 rows 2025-01-02 → 2025-08-29
[Semiconductors] saved SMH: 165 rows 2025-01-02 → 2025-08-29


In [35]:
def build_sector_daily_agg_from_stooq(tickers, start, end, out_path: Path):
    """
    Outer-join per-ticker series on date (UNION of trading days).
    Equal-weight close across available tickers. Sum volume across tickers.
    Adds volume features (vol_4w, vol_growth, vol_z). Also ret_1d and ret_fwd_5d.
    """
    frames = []
    for t in tickers:
        p = RAW_PRICES_DIR / f"{t}.csv"
        if not p.exists():
            print(f"[WARN] missing {p}")
            continue
        df = pd.read_csv(p, parse_dates=["date"])
        df = df[(df["date"] >= pd.to_datetime(start)) & (df["date"] <= pd.to_datetime(end))]
        df = df[["date","close","volume"]].rename(columns={"close": f"close_{t}", "volume": f"vol_{t}"})
        frames.append(df)

    if not frames:
        raise FileNotFoundError("No per-ticker price files found for this sector.")

    # Outer-join on date to keep UNION of all trading days
    agg = frames[0]
    for f in frames[1:]:
        agg = agg.merge(f, on="date", how="outer")

    agg = agg.sort_values("date").reset_index(drop=True)

    # Close: equal-weight mean across available tickers (skip NaN)
    close_cols = [c for c in agg.columns if c.startswith("close_")]
    agg["close_mean"] = agg[close_cols].mean(axis=1, skipna=True)

    # Volume: sum across available tickers (skip NaN). If you prefer equal-weight avg, change to .mean().
    vol_cols = [c for c in agg.columns if c.startswith("vol_")]
    if vol_cols:
        agg["volume_sum"] = agg[vol_cols].sum(axis=1, skipna=True)
        # Handle all-NaN rows (no ticker traded) — drop those dates
        agg = agg.dropna(subset=["close_mean"])
        agg["volume_sum"] = agg["volume_sum"].fillna(0)
    else:
        agg = agg.dropna(subset=["close_mean"])
        agg["volume_sum"] = 0.0

    # Returns
    agg["ret_1d"] = agg["close_mean"].pct_change()
    agg["ret_fwd_5d"] = agg["close_mean"].shift(-5)/agg["close_mean"] - 1

    # Volume engineered features (use log volume to stabilize)
    lv = np.log1p(agg["volume_sum"])
    agg["vol_4w"] = agg["volume_sum"].rolling(20, min_periods=5).sum()
    agg["vol_growth"] = agg["volume_sum"].pct_change()
    roll_mean = lv.rolling(126, min_periods=30).mean()
    roll_std  = lv.rolling(126, min_periods=30).std()
    agg["vol_z"] = (lv - roll_mean) / roll_std

    agg = agg[["date","close_mean","ret_1d","ret_fwd_5d","volume_sum","vol_4w","vol_growth","vol_z"]]
    agg.to_csv(out_path, index=False)
    print(f"Wrote {out_path} | rows={len(agg)} | dates {agg['date'].min().date()} → {agg['date'].max().date()}")

# Rebuild sector aggregates (union of dates + volume features)
for sector, meta in SECTOR_MAP.items():
    out = RAW_PRICES_DIR / f"sector_{sector}_daily_agg.csv"
    build_sector_daily_agg_from_stooq(meta["tickers"], START_DATE, END_DATE, out)


Wrote data/raw/prices/sector_Biotech_daily_agg.csv | rows=165 | dates 2025-01-02 → 2025-08-29
Wrote data/raw/prices/sector_Semiconductors_daily_agg.csv | rows=165 | dates 2025-01-02 → 2025-08-29
